In [1]:
!pip install konlpy
!pip install pymysql

  Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
  Using cached JPype1-1.4.0-cp38-cp38-macosx_10_9_x86_64.whl (381 kB)
  Using cached lxml-4.9.1-cp38-cp38-macosx_10_15_x86_64.whl (4.6 MB)
  Using cached numpy-1.23.1-cp38-cp38-macosx_10_9_x86_64.whl (18.1 MB)
  Using cached PyMySQL-1.0.2-py3-none-any.whl (43 kB)


In [4]:
import pandas as pd
import pymysql

conn = pymysql.connect(
                        user    = 'stocks',
                        passwd  = 'Stocks!',
                        host    = "54.204.204.186",
                        port    = 3306,
                        db      = 'Data',
                        charset = 'utf8'
        )

cur = conn.cursor()

cur.execute('''
SELECT * FROM Stock_Price
''')
result = cur.fetchall()

df = pd.DataFrame(result, columns=['stock_id', 'date', 'closing_price', 'market_price', 'high_price', 'low_price'])

df

,stock_id,date,closing_price,market_price,high_price,low_price
0,000060,2022-07-15,30800,31050,31650,30450
1,000060,2022-07-14,31650,32100,32350,31300
2,000060,2022-07-13,32100,32800,33000,31600
3,000060,2022-07-12,32550,32500,33150,32250
4,000060,2022-07-11,32550,32500,33200,32450
...,...,...,...,...,...,...
14795,375500,2022-03-31,132500,132000,133000,131500
14796,377300,2022-03-31,148500,150000,152000,147000
14797,381970,2022-03-31,32650,30900,33900,30900
14798,383220,2022-03-31,745000,726000,762000,726000


In [5]:
cur.execute('''
SELECT * FROM Stock_News
''')
news = cur.fetchall()
news = pd.DataFrame(news, columns=['id', 'stock_id', 'text', 'date'])
news

,id,stock_id,text,date
0,1,000060,2500억 모집액 못 채우고 미달\n3년물 2200억에 1710억 모집…5년물 전액...,2022-07-15
1,2,000060,메리츠증권은 14일 아모레G에 대해 아모레퍼시픽 부진으로 투자 매력도가 상대적으로 ...,2022-07-14
2,3,000060,증시 과매도…低PER 종목 쏟아진다\n30개社 PER 1~2배로 떨어져\n\nPER...,2022-07-13
3,4,000060,"투자의견 ‘매수’, 목표주가 7만5000원 유지\n사진=연합뉴스\n[파이낸셜뉴스] ...",2022-07-13
4,5,000060,"금감원, 손보사에 유사암·일반암 진단비 자료 요구\n올해들어 손보사 유사암 경쟁…진...",2022-07-13
...,...,...,...,...
154404,154917,402340,앱출석 후 회원가입시\n80만명 한정 선착순[헤럴드경제=박이담 기자]국내 최초 가상...,2022-04-01
154405,154918,402340,SK그룹 투자회사 SK스퀘어의 연결대상 종속회사 원스토어와 SK쉴더스가 증권신고서를...,2022-04-01
154406,154919,402340,31일 증권신고서 제출…4월 기관 수요예측\n\n\n[파이낸셜뉴스] SK스퀘어 자회...,2022-04-01
154407,154920,402340,[서울경제]\n\n<유가증권>\n\n▲삼부토건(001470)=제3자 배정증자 방식으...,2022-04-01


In [6]:
import re

def clean_text(d):
  text = re.sub(r'\([^)]*\)', '', d)
  text = re.sub(r'\[[^]]*\]', '', text)
  text = re.sub(r'\<[^>]*\>', '', text)
  pattern = r'[^가-힣0-9a-zA-Z\s]'
  text = re.sub(pattern, ' ', text)
  text = re.sub(r'사진', ' ', text)
  text = re.sub(r'.*뉴스', ' ', text)
  text = re.sub("\n", ' ', text)
  text = re.sub("  +", " ", text)
  return text

In [7]:
news['text'] = news.text.apply(clean_text)
news

,id,stock_id,text,date
0,1,000060,2500억 모집액 못 채우고 미달 3년물 2200억에 1710억 모집 5년물 전액 ...,2022-07-15
1,2,000060,메리츠증권은 14일 아모레G에 대해 아모레퍼시픽 부진으로 투자 매력도가 상대적으로 ...,2022-07-14
2,3,000060,증시 과매도 PER 종목 쏟아진다 30개 PER 1 2배로 떨어져 PER 낮은 기업...,2022-07-13
3,4,000060,투자의견 매수 목표주가 7만5000원 유지 메리츠증권이 13일 SK텔레콤에 대해 2...,2022-07-13
4,5,000060,금감원 손보사에 유사암 일반암 진단비 자료 요구 올해들어 손보사 유사암 경쟁 진단비...,2022-07-13
...,...,...,...,...
154404,154917,402340,앱출석 후 회원가입시 80만명 한정 선착순국내 최초 가상자산 거래소 코빗이 4월 한...,2022-04-01
154405,154918,402340,SK그룹 투자회사 SK스퀘어의 연결대상 종속회사 원스토어와 SK쉴더스가 증권신고서를...,2022-04-01
154406,154919,402340,31일 증권신고서 제출 4월 기관 수요예측 SK스퀘어 자회사들인 토종 앱스토어 원스...,2022-04-01
154407,154920,402340,삼부토건 제3자 배정증자 방식으로 SK에코플랜트에 보통주 11만 6106주 유상증...,2022-04-01


In [9]:
from konlpy.tag import Kkma
from tqdm import tqdm
tagger = Kkma()

In [10]:
token = [tagger.morphs(x) for x in tqdm(news['text'])]  
token

  0%|          | 143/154409 [02:19<33:44:15,  1.27it/s]